Третий принцип мы уже частично разбирали две недели назад. Это принцип подстановки Лисков.

Пусть есть свойство $P(x)$, где $x$ — некоторый класс. Тогда если $y$ — подкласс $x$, то $P(y)$.
Другими словами: если заменить объект $A$ на объект $B$, являющийся подтипом $A$, то никто ничего не заметит.

Классический пример нарушения LSP — [проблема квадрата-прямоугольника](https://habr.com/ru/post/351730/). С одной стороны любой квадрат является прямоугольником, но это нарушает LSP: у прямоугольника есть свойство "можно поменять одну сторону", которого нет у квадрата.

В конкретно этом случае надо очень хорошо подумать, а так ли нам надо разделять квадрат и прямоугольник на два разных класса и так ли нам надо их друг от друга наследовать. Может, и не надо на самом деле. Может, нам вообще хватит многоугольников. Или функции `is_square` (если нам надо только иногда обрабатывать запрос "выведи все квадраты и только их").

Чуть более тонкий пример нарушения LSP приведён ниже:

In [1]:
class mydict:
    def __init__(self):
        self.dict = {}
        
    def __getitem__(self, key):
        return self.dict[key]
    
    def __setitem__(self, key, value):
        assert key not in self
        self.dict[key] = value
        
    def __contains__(self, key):
        return key in self.dict

In [2]:
x = mydict()
x['foo'] = 20
x['bar'] = 30
print(x.dict)

{'foo': 20, 'bar': 30}


In [3]:
x['bar'] = 40

AssertionError: 

Если сказать, что `mydict` — подтип `dict`, то нарушится LSP. Дело в том, что у `dict` есть свойство "любой элемент можно переприсвоить". А у `mydict` однажды присвоенный элемент изменить нельзя.

Проблемы могут возникать неожиданно. Пусть есть функция `add_keys(d, keys, value)`, которая делает так, что в `d` появляются все ключи из `keys`. Если какого-то ключа не было, ему ставят в соответствие `value`:

In [4]:
def add_keys(d, keys, value):
    for key in keys:
        if key not in d:
            d[key] = value

In [5]:
d = {'foo': 10, 'baz': 20}
add_keys(d, ['foo', 'bar', 'baz'], 100)
print(d)

{'foo': 10, 'baz': 20, 'bar': 100}


In [6]:
d = mydict()
d['foo'] = 10
d['baz'] = 20
add_keys(d, ['foo', 'bar', 'baz'], 100)
print(d.dict)

{'foo': 10, 'baz': 20, 'bar': 100}


Кажется, что всё работает. Но мы могли бы реализовать функцию `add_keys` и по-другому! Не то чтобы красивее, просто по-другому:

In [7]:
def add_keys(d, keys, default_value):
    for key in keys:
        cur_value = default_value
        if key in d:
            cur_value = d[key]
        d[key] = cur_value

In [8]:
d = {'foo': 10, 'baz': 20}
add_keys(d, ['foo', 'bar', 'baz'], 100)
print(d)

{'foo': 10, 'baz': 20, 'bar': 100}


In [9]:
d = mydict()
d['foo'] = 10
d['baz'] = 20
add_keys(d, ['foo', 'bar', 'baz'], 100)
print(d.dict)

AssertionError: 

И в этот момент всё с треском падает. В новой реализации `add_keys` мы воспользовались предположением, что в словаре можно перезаписать элемент на такой же и ничего не произойдёт. Поведение `add_keys` с обычным словарём не поменялось. А в случае с `mydict` поменялось катастрофически.

Разумеется, здесь можно впасть в философию: а точно ли в контракте `add_keys` не было требования "не переприсваивать уже существующие элементы"?

Может и было, но тогда это стоит явно формулировать. Способ получше — явно сказать, кого ожидает `add_keys` и с какими требованиями, а когда говорим "подтип" (вроде `mydict`), пытаться найти способ сломать принцип подстановки Лисков. Если получилось — это не очень хорошо.

На бытовом уровне: всегда чётко формулируйте требования к интерфейсу/классу, а когда делаете подтип — строго их соблюдайте. А когда где-то используете интерфейс/класс — используйте только то, что описано в требованиях.